In [1]:
import numpy as np
import scipy.constants as sc
import matplotlib.pyplot as plt
from ase.io import read
import nwchem_nmode_parser as nmode
import nwchem_movecs_parser as nwcparse
import os
import nwchem_zora_parser as zora
# import importlib


In [2]:
# os.chdir("../")
os.getcwd()

'/mnt/Data/uday/work/isr/nwchem/dntt/spinphonon/dnss'

In [3]:
nm = nmode.ReadNmode('./0.02/NMODE_mw.nmode')
cminToeV = sc.h * sc.c * 100 / sc.e 
muB = sc.physical_constants['Bohr magneton in eV/T'][0]
delta = 1 * cminToeV


In [4]:
vibEval = nm.vibEval[6:]

vibInd=[]; 
Ez = [0.3 * 2 * muB]
for m in Ez:
#     b.extend( [[ind, jind] for ind, i in enumerate(vibEval) for jind, j in enumerate(vibEval) if [jind,ind] not in b and [ind,jind] and abs((j-i)-m) < delta and i != j  ])
    vibInd.extend( [[ind, jind] for ind, i in enumerate(vibEval) for jind, j in enumerate(vibEval) if abs(abs(j-i)-m) < delta and i!=j ] )

# vibInd=np.array(list(set(tuple(i) for i in b)))#.flatten()
del vibInd[1::2]
# vibInd = np.array(b)
vibInd = np.array(vibInd)
l = np.array(tuple(set(vibInd.flatten())))

l

array([ 19,  20,  28,  29,  35,  36,  37,  38,  41,  42,  43,  47,  48,
        51,  52,  53,  54,  56,  57,  59,  60,  63,  64,  67,  68,  82,
        83,  84,  85,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100, 101])

In [5]:
def SOCBasisTransform(mov,zora,PTO=1):
    psia = np.matrix(mov.psia); psib = np.matrix(mov.psib)
    Lx = zora.get("so_x"); Ly = zora.get("so_y"); Lz = zora.get("so_z")
    L_plus = Lx + 1j*Ly; L_minus = Lx - 1j*Ly
    H_ud = np.matmul(psia.getH(),np.matmul(L_minus,psib))
    H_uu = np.matmul(psia.getH(),np.matmul(Lz,psia))
    if PTO==2:
        return np.array([H_uu,H_ud])
    else:
        return (H_ud)
    
def calc_spinphononCART(d,PTO=2):
    xyz = read('vib.eq.xyz')
    nat = xyz.get_number_of_atoms()
    spinphononcart=[]
    if PTO==2:
        for a in range(nat):
            for i in 'xyz':
                zora_f = zora.read_nwchem_zora('vib.'+str(a)+str(i)+'+_sodft.zora_so')
                movecs_f = nwcparse.read('vib.'+str(a)+str(i)+'+_dft.movecs')
                soc_f = SOCBasisTransform(movecs_f,zora_f,PTO=2)
                zora_b = zora.read_nwchem_zora('vib.'+str(a)+str(i)+'-_sodft.zora_so')
                movecs_b = nwcparse.read('vib.'+str(a)+str(i)+'-_dft.movecs')
                soc_b = SOCBasisTransform(movecs_b,zora_b,PTO=2)
                sp = 0.5 * (soc_f - soc_b) / d
                spinphononcart.append(sp)
        spinphononcart = np.array(spinphononcart)
    np.save('SP_cart.npy',spinphononcart)

    
def spcart2norm(spcart):
    xyz = read('vib.eq.xyz')
    m = xyz.get_masses()
    nat = xyz.get_number_of_atoms()
    nm = nmode.ReadNmode('NMODE_mw.nmode')
    amu=sc.physical_constants['atomic mass constant'][0]
    evtoHz = sc.physical_constants['electron volt-hertz relationship'][0]
    HtoeV = sc.physical_constants['Hartree energy in eV'][0]
    vibEval = nm.vibEval
    vibEvec = np.reshape(nm.vibEvec,(nat*3,nat,3))
    nmo = 884
    spnorm = np.zeros((3*nat,2,nmo,nmo),dtype=np.complex128)
    spcart = np.reshape(spcart,(nat,3,2,nmo,nmo))
    
    for l in range(3*nat):
        for a in range(nat):
            for i in range(3):
                spnorm[l] += spcart[a,i] * vibEvec[l,a,i] * np.sqrt(sc.hbar/(2*m[a]*amu*vibEval[l]*evtoHz)) * 1e10 * HtoeV
        np.save('SP_norm.npy',spnorm)
    return spnorm
    

In [20]:
os.chdir("../")
os.getcwd()

'/mnt/Data/uday/work/isr/nwchem/dntt/spinphonon/dnss'

In [21]:
# disp = [0.005,0.006,0.007,0.008,0.009,0.01,0.02,0.03,0.04,0.05]
# disp = [0.006,0.008,0.02,0.04]
disp = [0.02]
path = os.getcwd()+"/"

spnormdisp=[]
for i in disp:
    print(i)
    subdir = path + str(i)
    os.chdir(subdir)
    if not os.path.isfile('SP_cart.npy'):
        calc_spinphononCART(d=i,PTO=2) 
        spnorm=spcart2norm(np.load('SP_cart.npy'))
    elif not os.path.isfile('SP_norm.npy'):
        spnorm = spcart2norm(np.load('SP_cart.npy'))
    else:
        spnorm = np.load('SP_norm.npy')
    spnormdisp.append(spnorm[6:])


0.02


In [22]:
spnormdisp = np.array(spnormdisp)
np.save('spnorm_disp.npy',spnormdisp)

In [23]:
l = np.array((vibInd)).flatten()
l

array([ 19,  20,  28,  29,  35,  36,  37,  38,  41,  42,  42,  41,  43,
        41,  47,  48,  51,  52,  53,  54,  56,  57,  59,  60,  63,  64,
        67,  68,  82,  83,  84,  85,  88,  89,  90,  91,  92,  93,  94,
        95,  96,  97,  98,  99, 100, 101])

In [7]:
spnormdisp = np.load('spnorm_disp.npy')

In [8]:
np.shape(spnormdisp)

(1, 102, 2, 884, 884)

In [9]:
disp=[0.02]
spph = []
for i in range(np.size(disp)):
    dictspph = {}
    for j in l:
        dictspph[j] = spnormdisp[i,j]
    spph.append(dictspph)

In [6]:
mov =  nwcparse.read('vib.eq_dft.movecs')
pi = sc.pi; 
hbar = sc.physical_constants['Planck constant over 2 pi in eV s'][0]
HtoeV = sc.physical_constants['Hartree energy in eV'][0]
cminToeV = sc.h * sc.c * 100 / sc.e 
vibEval = nm.vibEval[6:] 
kB = sc.physical_constants['Boltzmann constant in eV/K'][0]
amu = sc.physical_constants['atomic mass constant'][0]
freq = sc.physical_constants['electron volt-hertz relationship'][0]
delta = 1 * cminToeV #Broadening for finding resonant modes
elEvalU = mov.evalsa * HtoeV
elEvalD = mov.evalsb * HtoeV
idxHomo = mov.homo_index[0]
elEvalDiff = (mov.evalsa[:] - mov.evalsa[idxHomo]) * HtoeV

gauss   = lambda fwhm,E : (1/(fwhm*np.sqrt(2*pi))) * np.exp(-(E**2/(2*fwhm**2))) # for gaussian fwhm = sig*(2*np.sqrt(2*np.log(2)))
lorentz = lambda fwhm,E : (1/pi) * (0.5*fwhm)/(E**2 + (0.5*fwhm)**2)
bose    = lambda E, T   : 1/(np.exp(E/(kB*T))-1)
mu = (14*12.0107**-1 + 2*32.065**-1 + 8*1.00794**-1)**-1 # reduced mass
muB = sc.physical_constants['Bohr magneton in eV/T'][0]
temp = np.arange(50,450,50)
homoInd = 61
Ez = [0.400 * 2 * muB]
temp

array([ 50, 100, 150, 200, 250, 300, 350, 400])

In [26]:
t1_disp=[]
# for j in range(10):
for i in range(np.size(disp)):
    dictSpPh = spph[i]
    t1=[]
    for t in temp:
        rate_ae = 0; rate_ea = 0
        for i in vibInd:
            M = np.matmul(dictSpPh[i[0]][0,idxHomo,idxHomo+1:]/elEvalDiff[idxHomo+1:], dictSpPh[i[1]][1,idxHomo+1:,idxHomo]) + \
                np.matmul(dictSpPh[i[0]][1,idxHomo,idxHomo+1:]/elEvalDiff[idxHomo+1:], dictSpPh[i[1]][0,idxHomo+1:,idxHomo])
            rate_ae += (2*pi/hbar) * (bose(vibEval[i[0]],t)) * (bose(vibEval[i[1]],t)+1) * gauss(1*cminToeV,vibEval[i[1]]-vibEval[i[0]] - Ez[0]) * \
                (M * M.conj()) 
            M = np.matmul(dictSpPh[i[0]][0,idxHomo,:idxHomo]/elEvalDiff[:idxHomo], dictSpPh[i[1]][1,:idxHomo,idxHomo]) + \
                np.matmul(dictSpPh[i[0]][1,idxHomo,:idxHomo]/elEvalDiff[:idxHomo], dictSpPh[i[1]][0,:idxHomo,idxHomo])
            rate_ea += (2*pi/hbar) * (bose(vibEval[i[0]],t)+1) * (bose(vibEval[i[1]],t)) * gauss(1*cminToeV,vibEval[i[1]]-vibEval[i[0]] - Ez[0]) * \
                (M * M.conj()) 
        t1.append((rate_ea+rate_ae)**-1)
    t1_disp.append(t1)

In [27]:
t1_disp = np.array(t1_disp)
t1_disp[0],temp



(array([3.88043226e-03+0.j, 7.18135092e-07+0.j, 3.93334845e-08+0.j,
        9.00087577e-09+0.j, 3.61596614e-09+0.j, 1.91543526e-09+0.j,
        1.18640392e-09+0.j, 8.10231564e-10+0.j]),
 array([ 50, 100, 150, 200, 250, 300, 350, 400]))

In [43]:
t1dat=np.vstack((temp,t1_disp[0]))
t1dat[0]
np.save('t1_dnss',t1dat)

In [39]:
exp = np.repeat(1e-7,np.size(disp))
exp

array([1.e-07])

In [47]:
%matplotlib widget
plt.title(r'B = 400 mT')

plt.xlabel(r'$\Delta$ Q [A]')
plt.ylabel(r'$T_1$ [s]')
plt.ylim(1e-7,1e-5)
plt.plot(disp,exp,label='exp')
plt.semilogy(disp,t1_disp[:,6],'.-',label='theory')
plt.legend()
plt.savefig('T1_disp_dntt.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/uday/.local/lib/python3.5/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [44]:
%matplotlib widget
# for i in range(6):
x=[300];y = [1e-7]
plt.title(r'DNSS; B = 400 mT')
plt.xlabel(r'Temp [K]')
plt.ylabel(r'$T_1$ [s]')
# plt.xlim(140,420)
# plt.ylim(1e-7,3e-4)
plt.semilogy(temp,t1_disp[0],'.-',label='theory')
plt.plot(x,y,'.',label='exp',color='red')
plt.legend()
plt.savefig('T1_temp_dntt.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/uday/.local/lib/python3.5/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [23]:
sp = np.load('./SP_norm.npy')[6:]

In [26]:
dictSpPh = {}
for j in l:
    dictSpPh[j] = sp[j]
    
rate_ae = []; rate_ea = []
evtocmInv = 1/cminToeV

for i in vibInd:
    M = np.matmul(dictSpPh[i[0]][0,idxHomo,idxHomo+1:]/elEvalDiff[idxHomo+1:], dictSpPh[i[1]][1,idxHomo+1:,idxHomo]) + \
        np.matmul(dictSpPh[i[0]][1,idxHomo,idxHomo+1:]/elEvalDiff[idxHomo+1:], dictSpPh[i[1]][0,idxHomo+1:,idxHomo])
    rate_ae.append(M * M.conj() * evtocmInv)
    M = np.matmul(dictSpPh[i[0]][0,idxHomo,:idxHomo]/elEvalDiff[:idxHomo], dictSpPh[i[1]][1,:idxHomo,idxHomo]) + \
        np.matmul(dictSpPh[i[0]][1,idxHomo,:idxHomo]/elEvalDiff[:idxHomo], dictSpPh[i[1]][0,:idxHomo,idxHomo])
    rate_ea.append(M * M.conj() * evtocmInv)

vibInd = np.array(vibInd)
veval  = [vibEval[i] for i in vibInd[:,0]]

In [29]:

%matplotlib widget

fig, ax = plt.subplots()
width=0.002

ax.set_yscale('log')
ax.set_ylim(1e-12,1e-4)
ax.set_xlim(-0.01,0.42)
ax.set_ylabel(r'|M$^{sp}|^2$  [cm$^{-2}$]')
ax.set_xlabel('Phonon Energy [eV]')
ax.set_title('DNSS')

ax.bar(veval, rate_ae, width, label='AE')
ax.bar(veval, rate_ea, width, bottom=rate_ae, label='EA')

plt.savefig('DNSS_matrix.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/uday/.local/lib/python3.5/site-packages/matplotlib/transforms.py:827: ComplexWarning: Casting complex values to real discards the imaginary part
  points = np.array(args, dtype=float).reshape(2, 2)
/home/uday/.local/lib/python3.5/site-packages/matplotlib/transforms.py:2010: ComplexWarning: Casting complex values to real discards the imaginary part
  x, y = float(x), float(y)
/home/uday/.local/lib/python3.5/site-packages/matplotlib/transforms.py:954: ComplexWarning: Casting complex values to real discards the imaginary part
  self._points[:, 1] = interval


In [31]:
np.shape(spnormdisp)

(1, 102, 2, 884, 884)

In [9]:
def calc_t1(disp,delta): # delta in cminv 
    broad = delta * cminToeV
    vibInd=[]; 
    Ez = [0.3 * 2 * muB]
#     for m in Ez:
#         vibInd.extend( [[ind, jind] for ind, i in enumerate(vibEval) for jind, j in enumerate(vibEval) if abs(abs(j-i)-m) < broad and i!=j ] )
    vibInd = np.array([[ind,jind] for ind,i in enumerate(vibEval) for jind, j in enumerate(vibEval) if jind > ind])
#     del vibInd[1::2]
    vibInd = np.array(vibInd)
    l = np.array(tuple(set(vibInd.flatten())))
    spph = []
    for i in range(np.size(disp)):
        dictspph = {}
        for j in l:
            dictspph[j] = spnormdisp[i,j]
        spph.append(dictspph)
        t1_disp=[]
    # for j in range(10):
    for i in range(np.size(disp)):
        dictSpPh = spph[i]
        t1=[]
        for t in temp:
            rate_ae = 0; rate_ea = 0
            for i in vibInd:
                M = np.matmul(dictSpPh[i[0]][0,idxHomo,idxHomo+1:]/elEvalDiff[idxHomo+1:], dictSpPh[i[1]][1,idxHomo+1:,idxHomo]) + \
                    np.matmul(dictSpPh[i[0]][1,idxHomo,idxHomo+1:]/elEvalDiff[idxHomo+1:], dictSpPh[i[1]][0,idxHomo+1:,idxHomo])
                rate_ae += (2*pi/hbar) * (bose(vibEval[i[0]],t)) * (bose(vibEval[i[1]],t)+1) * gauss(broad,vibEval[i[1]]-vibEval[i[0]] - Ez[0]) * \
                    (M * M.conj()) 
                M = np.matmul(dictSpPh[i[0]][0,idxHomo,:idxHomo]/elEvalDiff[:idxHomo], dictSpPh[i[1]][1,:idxHomo,idxHomo]) + \
                    np.matmul(dictSpPh[i[0]][1,idxHomo,:idxHomo]/elEvalDiff[:idxHomo], dictSpPh[i[1]][0,:idxHomo,idxHomo])
                rate_ea += (2*pi/hbar) * (bose(vibEval[i[0]],t)+1) * (bose(vibEval[i[1]],t)) * gauss(broad,vibEval[i[1]]-vibEval[i[0]] - Ez[0]) * \
                    (M * M.conj()) 
            t1.append((rate_ea+rate_ae)**-1)
        t1_disp.append(t1)
#         t1_disp = np.array(t1_disp)
    return t1_disp

In [10]:
disp=[0.01]
delr=[0.3,0.5,1,2,5,10,100]
t1_del = []
for i in delr:
    t1_del.append(calc_t1(disp,i))
# t1_del = np.array(t1_del)

In [11]:
%matplotlib widget
# for i in range(6):
x=[300];y = [1e-7]
plt.title(r'DNSS; B = 0.3 T')
plt.xlabel(r'Temp [K]')
plt.ylabel(r'$T_1$ [s]')
# plt.xlim(140,420)
# plt.ylim(1e-9,1e3)
# exp = np.loadtxt('vopc_exp.txt')
plt.semilogy(x,y,'.',label='Exp')

for ind, i in enumerate(delr):
    plt.semilogy(temp,t1_del[ind][0],'-',label=r'$\delta=$'+str(i)+r' cm$^{-1}$')
    
plt.legend()
plt.savefig('T1_temp_DNSS.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/uday/.local/lib/python3.5/site-packages/matplotlib/axes/_base.py:3507: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=1e-07, top=1e-07
  self.set_ylim(upper, lower, auto=None)
/home/uday/.local/lib/python3.5/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
